In [25]:
import matplotlib.pyplot as plt 
%matplotlib inline
import metapack as mp
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from rowgenerators import RowGenerator
from sdipylib.text import make_acro
from csv import reader, writer
from os.path import exists
import fuzzy



In [26]:
src = mp.jupyter.open_source_package()
doc = mp.jupyter.open_package()
doc

In [14]:

def clean_comm_name(s):
    return re.compile('[^a-zA-Z]').sub('',s.lower())  


if True:

    import re
    
    c = mp.jupyter.open_package().resource('communities')
    
    communities = sorted(set([ c['name'] for c in c.iterdict if c['type'] == 'sd_community' or c['type'] == 'county_community']    ))

    fn = '../data/community-acronyms.csv'

    rows = []

    if exists(fn):
        with open(fn) as f:
            r = reader(f)
            headers = next(r)
            rows = list(r)

    names = dict(rows)
    past = set(e[1] for e in rows) 

    with open(fn, 'w') as f:
        w = writer(f)

        w.writerow(['name', 'code'])
        
        for community in communities:

            community = clean_comm_name(community)
            
            if community in names: 
                print("Existing name", community)
                acro = names[community]
            elif community.strip():
                acro = make_acro(past, '', community)
            else:
                continue # Empty

            w.writerow([community,acro])


In [47]:

tc = doc.resource('tracts_all_regions').dataframe()
tracts = doc.resource('tracts').geoframe()
acronyms = src.reference('acronyms')

acro_map = dict(list(acronyms)[1:])
acro_map[''] = ''

_1 = tc[['geoid']].drop_duplicates().set_index('geoid').join(tracts[['geoid', 'geometry']].set_index('geoid'))
_2 = tc.set_index('geoid')

_3  = _2[_2.type == 'city'][['name','name_code']]
_4  = _2[_2.type == 'county'][['name','name_code']]
_5  = _2[_2.type == 'sd_community'][['name','name_code']]
_6  = _2[_2.type == 'county_community'][['name','name_code']]

_7 = _1.join(_3, rsuffix='_city').join(_4, rsuffix='_county')\
       .join(_5, rsuffix='_sdc').join(_6, rsuffix='_cnc')

_7.columns =\
[
 'geometry',
 'city_name',
 'city_code',
 'county_name',
 'county_code',
 'community_name',
 'community_cpcode',
 'cnc_name',
 'cnc_code']
  
# Move the county name into the city columns, then drop it
_7['city_name'] = _7.city_name.where(~((_7.city_name.isnull()) & (_7.county_code == 'CN')),'COUNTY' )
_7['city_code'] = _7.city_code.where(~((_7.city_code.isnull()) & (_7.county_code == 'CN')),'CN' )

_7.drop(['county_name','county_code'], axis=1, inplace=True)
   

# Move the  county community names into the community columns
_7['community_name'] =  _7.community_name.where( _7.city_code != 'CN' ,  _7.cnc_name ).fillna('')
_7['community_cpcode'] =_7.community_cpcode.where( _7.city_code != 'CN' ,  _7.cnc_code ).fillna('0')

_7.drop(['cnc_name','cnc_code'], axis=1, inplace=True)

_7 = _7.fillna('')
    
_7['city_name'] = _7.city_name.apply(lambda v : v.title())
_7['community_name'] = _7.community_name.apply(lambda v : str(v).title()).fillna('')

_7['community_code'] = _7.community_name.apply(lambda v: acro_map[clean_comm_name(v)].upper() )

 # move geometry to the end
_7 = _7[ list(_7.columns)[1:]+ list(_7.columns)[:1]]

_7.head().T

geoid,14000US06073020309,14000US06073020207,14000US06073020211,14000US06073020209,14000US06073020206
city_name,County,Escondido,Escondido,Escondido,Escondido
city_code,CN,ES,ES,ES,ES
community_name,North County Metro,,,,
community_cpcode,190900,0,0,0,0
community_code,NOR,,,,
geometry,"POLYGON ((-117.121465 33.132039, -117.121422 3...","POLYGON ((-117.060258 33.135246, -117.060224 3...","POLYGON ((-117.072943 33.139046, -117.072574 3...","POLYGON ((-117.053329 33.140472, -117.0532 33....","POLYGON ((-117.065902 33.143224, -117.065549 3..."
